In [1]:
import uuid

class UTXO:
    def __init__(self, txid, amount, owner):
        self.id = txid  # Unique UTXO ID
        self.amount = amount
        self.owner = owner

class Transaction:
    def __init__(self, inputs, outputs, fee):
        self.id = str(uuid.uuid4())
        self.inputs = inputs      # List of UTXO ids
        self.outputs = outputs    # List of UTXO objects
        self.fee = fee

class Blockchain:
    def __init__(self):
        self.utxos = {}   # utxo_id: UTXO
        self.chain = []   # List of transactions

    def add_utxo(self, utxo):
        self.utxos[utxo.id] = utxo

    def get_balance(self, owner):
        return sum(utxo.amount for utxo in self.utxos.values() if utxo.owner == owner)

    def get_utxos(self, owner):
        return [utxo for utxo in self.utxos.values() if utxo.owner == owner]

    def make_transaction(self, sender, recipients, fee):
        utxos = self.get_utxos(sender)
        utxos.sort(key=lambda x: x.amount, reverse=True)

        total_needed = sum(amount for _, amount in recipients) + fee
        selected = []
        total = 0

        for utxo in utxos:
            selected.append(utxo)
            total += utxo.amount
            if total >= total_needed:
                break

        if total < total_needed:
            raise ValueError(f"Insufficient balance: needed {total_needed}, found {total}")

        # Build inputs and outputs
        input_ids = [u.id for u in selected]
        outputs = [UTXO(str(uuid.uuid4()), amt, recv) for recv, amt in recipients]

        change = total - total_needed
        if change > 0:
            outputs.append(UTXO(str(uuid.uuid4()), change, sender))

        tx = Transaction(input_ids, outputs, fee)
        self.chain.append(tx)

        # Update UTXO set
        for txid in input_ids:
            del self.utxos[txid]

        for utxo in outputs:
            self.utxos[utxo.id] = utxo

        return tx

    def print_chain(self):
        for tx in self.chain:
            print(f"\nTransaction {tx.id}")
            print(f"  Inputs: {tx.inputs}")
            print(f"  Fee: {tx.fee} BTC")
            for out in tx.outputs:
                print(f"  Output: {out.amount} BTC -> {out.owner}")

# ----------------------
# Simulation
# ----------------------

bc = Blockchain()

# Initial UTXOs
bc.add_utxo(UTXO("imran1", 2.0, "Jawad"))
bc.add_utxo(UTXO("farooq1", 0.4, "Jawad"))
bc.add_utxo(UTXO("mohsin1", 0.6, "Jawad"))
bc.add_utxo(UTXO("mujtaba1", 0.2, "Jawad"))

# Amazon: Laptop
bc.make_transaction("Jawad", [("Amazon", 1.3)], fee=0.1)

# Amazon: HardDisk
bc.make_transaction("Jawad", [("Amazon", 0.2)], fee=0.1)

# Yasir pays 3.4 BTC to Jawad
bc.add_utxo(UTXO("yasir1", 3.4, "Jawad"))

# Alibaba: Bicycle
bc.make_transaction("Jawad", [("Alibaba", 2.1)], fee=0.2)

# Alibaba: Printer
bc.make_transaction("Jawad", [("Alibaba", 0.3)], fee=0.1)

# Final blockchain and Jawad's balance
print("\n========== Blockchain ==========")
bc.print_chain()

print("\n========== Jawad's Final Balance ==========")
print(f"Jawad's Balance: {bc.get_balance('Jawad')} BTC")

print("\n========== Jawad's UTXOs ==========")
for utxo in bc.get_utxos("Jawad"):
    print(f"{utxo.id}: {utxo.amount} BTC")



========== Blockchain ==========

Transaction 5273a92e-4e95-4ca5-9d80-45760c7cdb8a
  Inputs: ['imran1']
  Fee: 0.1 BTC
  Output: 1.3 BTC -> Amazon
  Output: 0.5999999999999999 BTC -> Jawad

Transaction 4ca350d1-2390-4e19-bc4f-56c22efc2879
  Inputs: ['mohsin1']
  Fee: 0.1 BTC
  Output: 0.2 BTC -> Amazon
  Output: 0.29999999999999993 BTC -> Jawad

Transaction 5c0cdbc7-4086-4736-9e76-85504582f9ce
  Inputs: ['yasir1']
  Fee: 0.2 BTC
  Output: 2.1 BTC -> Alibaba
  Output: 1.0999999999999996 BTC -> Jawad

Transaction 4bd56115-1199-4c99-a311-622558c0ba86
  Inputs: ['a0574be1-83d0-4075-ae09-2bf4df6c9546']
  Fee: 0.1 BTC
  Output: 0.3 BTC -> Alibaba
  Output: 0.6999999999999996 BTC -> Jawad

========== Jawad's Final Balance ==========
Jawad's Balance: 2.1999999999999997 BTC

========== Jawad's UTXOs ==========
farooq1: 0.4 BTC
mujtaba1: 0.2 BTC
242a20dd-e68b-4a5f-8957-27f3d8acb545: 0.5999999999999999 BTC
ef3b293c-1290-4ca3-9d57-78ddb628bb03: 0.29999999999999993 BTC
a3cb71af-2dd0-42c4-a9dd-9634